In [1]:
import os, sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import xml.etree.ElementTree as ET
import html.parser
from xml.sax.saxutils import unescape
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','<','br','>','Q','A'])
doc = open('CutData/Pseudo/NTF_200/Features_NTF2_1/Doc_NTF2_1.csv', 'r').read() 
doc_num = doc.split('\n')  
tree = ET.parse('Formspring/new_XMLMergedFile.xml')
root = tree.getroot()

In [3]:
import csv
def store(data,data2,data3):
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/Senti_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['SentiPos(num)','SentiNeg(num)','Senti(score)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line,line2,line3 in zip(data,data2,data3):
            writer.writerow({'SentiPos(num)': str(line),'SentiNeg(num)': str(line2),'Senti(score)': str(line3)})

In [4]:
total_sentiscore_list=[]
pos_count_list=[]
neg_count_list=[]
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        stokens = nltk.word_tokenize(content)#斷詞
        filtered_sentence = [w for w in stokens if not w in stop_words]#刪除停用詞
        taggedlist=nltk.pos_tag(filtered_sentence)#詞性標註
        wnl = nltk.WordNetLemmatizer() #做詞性還原
        score_list = []
        for t in taggedlist:
            lemmatized = wnl.lemmatize(t[0])
            newtag=''
            if t[1].startswith('NN'):
                newtag='n'
            elif t[1].startswith('JJ'):
                newtag='a'
            elif t[1].startswith('VB'):
                newtag='v'
            elif t[1].startswith('RB'):
                newtag='r'
            else:
                newtag=''       
            if(newtag != ''):
                synsets = list(swn.senti_synsets(lemmatized, newtag))
               # print(lemmatized, newtag)
                score = 0
                if(len(synsets) > 0):
                    nosenti_count = 0
                    for syn in synsets:
                        if syn.pos_score() == 0 and syn.neg_score() == 0:
                            nosenti_count += 1
                            continue
                        else:
                            score +=syn.pos_score()-syn.neg_score()       
                    if len(synsets) != nosenti_count:
                        score_list.append(score/(len(synsets)-nosenti_count)) 
                     #   print(score_list)
                else:
                    continue
        if score_list != []:
            total_sentiscore = 0
            pos_count = 0
            neg_count = 0
            for score in score_list:
                if score >= 0:
                    pos_count += 1
                else:
                    neg_count += 1
                total_sentiscore += score
            total_sentiscore_list.append(total_sentiscore)
            pos_count_list.append(pos_count)
            neg_count_list.append(neg_count)
        else:
            total_sentiscore_list.append(0)
            pos_count_list.append(0)
            neg_count_list.append(0)
store(pos_count_list,neg_count_list,total_sentiscore_list)
    #print('情感分數:',total_sentiscore,'正向情感個數',pos_count,'負向情感個數',neg_count)